In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon May 8 2023
Calculate mass balance of Hg in Amazon from simulations
@author: arifeinberg
"""

'\nCreated on Mon May 8 2023\nCalculate mass balance of Hg in Amazon from simulations\n@author: arifeinberg\n'

In [8]:
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import scipy.io as sio
import datetime
import xarray as xr
import sys
sys.path.append("/home/jovyan/colomb_ige-m2-internship-2025_MERCAMAZON-project/scripts/feinberg")
from helper_functions import ds_sel_yr, annual_avg, open_Hg

In [9]:
# Load grid cell area for unit conversion of model
fn_gbox = '/mnt/data-summer-user/analysis_scripts/misc_Data/GEOSChem_2x25_gboxarea.nc'
ds_gbox = xr.open_dataset(fn_gbox)
gbox_GC = ds_gbox.cell_area #m2

# Load Amazon mask
fn_Am_mask = '/mnt/data-summer-user/analysis_scripts/misc_Data/Amazon_basin_mask_2x25.nc'
ds_Am_mask = xr.open_dataset(fn_Am_mask)
Am_mask = ds_Am_mask.MASK #unitless

In [12]:
sim = ['BAU','HIST']
#sim = ['0300','0302','0301','0304']
sim_name = ['BAU','HIST']
#Erosion = np.array([63.6,84.5, 72.6,  124.7])


In [11]:

# initialize matrices
Emiss_tot = np.zeros(len(sim))
Am_bb_tot = np.zeros(len(sim))
Am_soil_tot = np.zeros(len(sim))
Am_dd_Hg0_tot = np.zeros(len(sim))
Dep_tot = np.zeros(len(sim))
Balance = np.zeros(len(sim))# run loop
for i in range(len(sim)):
    sim_i = sim[i]
    fn_emis = '/mnt/data-summer-user/src/data_feinberg/Feinberg23_deforest_Hg/GC_data/run'+sim_i+'/GEOSChem.MercuryEmis.2015_m.nc4'
    ds_emis = xr.open_dataset(fn_emis)
    
    soil_emis_yr = ds_sel_yr(ds_emis, 'EmisHg0soil', 2015) 
    land_emis_yr = ds_sel_yr(ds_emis, 'EmisHg0land', 2015) 
    geogen_emis_yr = ds_sel_yr(ds_emis, 'EmisHg0geogenic', 2015) 
    bb_emis_yr = ds_sel_yr(ds_emis, 'EmisHg0biomass', 2015) 
    ant0_emis_yr = ds_sel_yr(ds_emis, 'EmisHg0anthro', 2015) 
    ant2_emis_yr = ds_sel_yr(ds_emis, 'EmisHg2HgPanthro', 2015) 
    
    # Convert model data from kg/s to kg/yr for annual average   
    s_in_yr = 365.2425 * 24 * 3600 # s in one year
    
    unit_conv = s_in_yr
        
    soil_emis = annual_avg(soil_emis_yr) * unit_conv # kg/yr
    land_emis = annual_avg(land_emis_yr) * unit_conv # kg/yr
    geogen_emis = annual_avg(geogen_emis_yr) * unit_conv # kg/yr
    bb_emis = annual_avg(bb_emis_yr) * unit_conv # kg/yr
    ant0_emis = annual_avg(ant0_emis_yr) * unit_conv # kg/yr
    ant2_emis = annual_avg(ant2_emis_yr) * unit_conv # kg/yr
    # Load deposition fluxes
    # dry dep
    fn_dd = '/mnt/data-summer-user/src/data_feinberg/Feinberg23_deforest_Hg/GC_data/run'+sim_i+'/GEOSChem.DryDep.2015_m.nc4'
    ds_dd = xr.open_dataset(fn_dd)
    
    dd_Hg0_yr = ds_sel_yr(ds_dd, 'DryDep_Hg0', 2015) 
    dd_Hg2_yr = ds_sel_yr(ds_dd, 'DryDep_Hg2', 2015) 
    dd_HgP_yr = ds_sel_yr(ds_dd, 'DryDep_HgP', 2015) 
    
    # Convert model data from molec/cm2/s to kg/yr for annual average   
    s_in_yr = 365.2425 * 24 * 3600 # s in one year
    g_kg = 1e-3 # g in kg
    cm2_m2 = 1e4 # cm^2 in m^2
    MW_Hg = 200.59 # g mol^-1
    avo = 6.02e23 # avogadro number molec mol^-1
    
    unit_conv = MW_Hg / avo * g_kg * cm2_m2 * s_in_yr * gbox_GC # constant to convert units
    
    dd_Hg0 = annual_avg(dd_Hg0_yr) * unit_conv # kg/yr
    dd_Hg2 = annual_avg(dd_Hg2_yr) * unit_conv # kg/yr
    dd_HgP = annual_avg(dd_HgP_yr) * unit_conv # kg/yr
    
    # wet dep
    fn_dd = '/mnt/data-summer-user/src/data_feinberg/Feinberg23_deforest_Hg/GC_data/run'+sim_i+'/GEOSChem.WetLossTotal.2015_m.nc4'
    ds_dd = xr.open_dataset(fn_dd)
    
    wd_Hg_yr = ds_sel_yr(ds_dd, 'WetLossTot_Hg', 2015) 
    
    # Convert model data from kg/s to kg/yr for annual average   
    s_in_yr = 365.2425 * 24 * 3600 # s in one year
    
    unit_conv = s_in_yr
    
    wd_Hg = annual_avg(wd_Hg_yr) * unit_conv # kg/yr
    
    # Calculate Amazon totals
    kg_Mg = 1e3
    
    # emissions
    Am_soil_tot[i] = ((soil_emis * Am_mask).sum() / kg_Mg).values # Mg/yr
    Am_land_tot = ((land_emis * Am_mask).sum() / kg_Mg).values # Mg/yr
    Am_geogen_tot = ((geogen_emis * Am_mask).sum() / kg_Mg).values # Mg/yr
    Am_bb_tot[i] = ((bb_emis * Am_mask).sum() / kg_Mg).values # Mg/yr
    Am_ant0_tot = ((ant0_emis * Am_mask).sum() / kg_Mg).values # Mg/yr
    Am_ant2_tot = ((ant2_emis * Am_mask).sum() / kg_Mg).values # Mg/yr
    
    Emiss_nat = Am_geogen_tot # natural
    Emiss_rec = Am_soil_tot[i] + Am_land_tot + Am_bb_tot[i] # legacy re-emissions
    Emiss_ant = Am_ant0_tot + Am_ant2_tot  # anthropogenic emissions
    
    Emiss_tot[i] = Emiss_nat + Emiss_rec + Emiss_ant

    # deposition
    Am_dd_Hg0_tot[i] = ((dd_Hg0 * Am_mask).sum() / kg_Mg).values # Mg/yr
    Am_dd_Hg2_tot = ((dd_Hg2 * Am_mask).sum() / kg_Mg).values # Mg/yr
    Am_dd_HgP_tot = ((dd_HgP * Am_mask).sum() / kg_Mg).values # Mg/yr
    
    Am_wd_Hg_tot = ((wd_Hg * Am_mask).sum() / kg_Mg).values # Mg/yr
    
    Dep_tot[i] = Am_wd_Hg_tot + Am_dd_Hg0_tot[i] + Am_dd_Hg2_tot + Am_dd_HgP_tot
    
    Balance[i] = Emiss_tot[i] - Dep_tot[i]
    # print values
    print(sim_i)
    print("Soil: " + str(Am_soil_tot[i]))
    print("Biomass burning: " + str(Am_bb_tot[i]))
    print("Land: " + str(Am_land_tot))
    print("Geogenic: " + str(Am_geogen_tot))
    print("Anthro Hg0: " + str(Am_ant0_tot))
    print("Anthro Hg2/P: " + str(Am_ant2_tot))
    
    print("dd Hg0: " + str(Am_dd_Hg0_tot[i]))
    print("dd Hg2: " + str(Am_dd_Hg2_tot))
    print("dd HgP: " + str(Am_dd_HgP_tot))
    print("wd Hg: " + str(Am_wd_Hg_tot))
    
    print("Total emiss: " + str(Emiss_tot[i]))
    print("Total dep: " + str(Dep_tot[i]))
#    print("Total erosion: " + str(Erosion[i]))
    
    print("Balance: " + str(Balance[i]))

KeysView(Frozen({'lat_bnds': <xarray.Variable (lat: 91, nb: 2)> Size: 1kB
[182 values with dtype=float64], 'lon_bnds': <xarray.Variable (lon: 144, nb: 2)> Size: 2kB
[288 values with dtype=float64], 'hyam': <xarray.Variable (lev: 47)> Size: 376B
[47 values with dtype=float64], 'hybm': <xarray.Variable (lev: 47)> Size: 376B
[47 values with dtype=float64], 'hyai': <xarray.Variable (ilev: 48)> Size: 384B
[48 values with dtype=float64], 'hybi': <xarray.Variable (ilev: 48)> Size: 384B
[48 values with dtype=float64], 'P0': <xarray.Variable ()> Size: 8B
[1 values with dtype=float64], 'AREA': <xarray.Variable (lat: 91, lon: 144)> Size: 52kB
[13104 values with dtype=float32], 'Hg2GasToSSA': <xarray.Variable (lev: 47, lat: 91, lon: 144)> Size: 2MB
[615888 values with dtype=float32], 'Hg2GasToHg2StrP': <xarray.Variable (lev: 47, lat: 91, lon: 144)> Size: 2MB
[615888 values with dtype=float32], 'Hg2PToHg2G': <xarray.Variable (lev: 47, lat: 91, lon: 144)> Size: 2MB
[615888 values with dtype=float32]